# Tarea5: Series de Taylor

Fecha de envío del PR inicial: **viernes 21 de abril**

Fecha de aceptación del PR: **martes 25 de abril, antes de la clase**

---

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests_taylor.jl") para cada uno de los métodos que implementen.


In [1]:
"""
Definición de polinomios de Taylor, donde
...
"""
type Taylor{T<:Number}
    orden::Int64 #el orden de la serie de taylor.
    coff::Array{T,1}   #coeficientes en la serie de taylor.
end

Taylor

In [2]:
function taylor(n,v)    #ESTE AGREGADO ES NECESARIO PARA GARANTIZAR QUE TODOS LOS ELEMENTOS INGRESADOS EN EL VECTOR SEAN DEL MISMO ORDEN QUE LA N PROPUESTA
    if length(v)==n
        return Taylor(n-1,v)   #el primer elemento del vector es de orden 0, por eso la sustraccción
    else
        error("función no válida")
    end
end

taylor (generic function with 1 method)

In [3]:
a=[2,3,4,5,1.2]#es necesario recordar que sólo reconoce vectores con aunque sea un valor de punto flotante

5-element Array{Float64,1}:
 2.0
 3.0
 4.0
 5.0
 1.2

In [4]:
a[4]

5.0

In [5]:
taylor(2,a)

LoadError: función no válida

In [6]:
a=[2,4]

2-element Array{Int64,1}:
 2
 4

In [7]:
taylor(2,a)

Taylor{Int64}(1,[2,4])

Esto sólo es un ejemplo para ver que el nuevo type *Taylor* funciona correctamente

In [8]:
function xTaylor(n)
    w=zeros(n) #genera un vector de n ceros
    w=deleteat!(w,n)#el último elemento no es cero
    w=append!(w,1.0) #el último elemento del vector de orden cero es 1.0
    return taylor(n,w)
end

xTaylor (generic function with 1 method)

In [9]:
xTaylor(2)

Taylor{Float64}(1,[0.0,1.0])

In [10]:
m=[1,2]

2-element Array{Int64,1}:
 1
 2

In [11]:
taylor(2,m)

Taylor{Int64}(1,[1,2])

In [12]:
n=ans

Taylor{Int64}(1,[1,2])

In [13]:
n.coff

2-element Array{Int64,1}:
 1
 2

In [14]:
q=taylor(2,[1,5])

Taylor{Int64}(1,[1,5])

In [15]:
n.coff+q.coff

2-element Array{Int64,1}:
 2
 7

In [16]:
size(n.coff)==size(q.coff)

true

In [17]:
function igualavector(a,b)    #esta funcion compara los valores de los órdenes de los polinomios
    if length(a.coff) < length(b.coff)
        for i in length(a.coff):length(b.coff)-1    
            push!(a.coff,0)
        end
        return alfa=b.orden     #sea cualquier caso, de no ser del mismo orden, el de orden menor aumenta su grado para ser igual al de grado mayor
    elseif length(a.coff) >length(b.coff)
        for i in length(b.coff):length(a.coff)-1
            push!(b.coff,0)
        end 
        return alfa=a.orden
    end
    return alfa=a.orden
end

igualavector (generic function with 1 method)

In [29]:
k=Taylor(2,[1,1])
j=Taylor(3,[0,1,3])

Taylor{Int64}(3,[0,1,3])

In [30]:
igualavector(j,k)

3

In [31]:
function sumavector(a,b)
    igualavector(a,b)    #condiciones quizá no tan necesarias, pero no funcionaban junto a la funcion igualavector
    v=a.coff+b.coff
    return v
end

sumavector (generic function with 1 method)

Any, Any) in module Main at In[20]:2 overwritten at In[31]:2.


In [32]:
sumavector(j,k)

3-element Array{Int64,1}:
 1
 2
 3

In [33]:
function restavector(a,b)  #análogo a la suma
    igualavector(a,b)
    v=a.coff-b.coff
    return v
end

restavector (generic function with 1 method)

Any, Any) in module Main at In[22]:2 overwritten at In[33]:2.


In [34]:
restavector(j,k)

3-element Array{Int64,1}:
 -1
  0
  3

Una vez definido la suma y la resta, falta programar la multiplicación (la división se puede obtener con un algoritmo semejante al de la multiplicación)

In [143]:
function multvector(v,w)
    n=length(v.coff)+length(w.coff)-1
    x=zeros(n) #genera un vector de orden w+v para almacenar los resultados
    a=length(v.coff)
    b=length(w.coff)
    l=1
    for j in 1:a   #primer for, va cambiando el multiplicador por cada elemento de v.
        for i in 1:b #segundo for, ejecuta la multiplicación con respecto a cada elemento de w.
            x[i]=x[i]+(v.coff[j])*(w.coff[i])
        end
    end
    return taylor(n,x)
end

multvector (generic function with 1 method)

Any) in module Main at In[140]:2 overwritten at In[143]:2.


In [43]:
w=[0,1,0,0]

4-element Array{Int64,1}:
 0
 1
 0
 0

In [44]:
w[2]=w[2]+1

2

In [144]:
k=taylor(2,[1,1])
j=taylor(2,[1,1])

Taylor{Int64}(1,[1,1])

In [145]:
multvector(k,j)

Taylor{Float64}(2,[2.0,2.0,0.0])

In [26]:
import Base: +, -, *, /, ==

# Aqui se implementan los métodos necesarios para cada función
+(A::Taylor,B::Taylor)=Taylor(igualavector(A,B),sumavector(A,B))
+(A::Taylor,B::Taylor)=Taylor(igualavector(A,B),A.coff+B.coff)  #en caso de que el orden sea el mismo, la suma es directa
-(A::Taylor,B::Taylor)=Taylor(igualavector(A,B),restavector(A,B))
-(A::Taylor,B::Taylor)=Taylor(igualavector(A,B),A.coff-B.coff) #en caso de que el orden sea el mismo, la resta es directa
*(A::Real,B::Taylor)=Taylor(B.orden,A*B.coff)
*(A::Taylor,B::Real)=Taylor(A.orden,B*A.coff)

/(A::Taylor,B::Real)=Taylor(A.orden,(A.coff)/B)

==(A::Taylor,B::Taylor)=(A.orden==B.orden && A.coff==B.coff)

== (generic function with 119 methods)

Taylor) in module Main at In[26]:4 overwritten at In[26]:5.


In [27]:
# Muestren que su código funciona con tests adecuados

using Base.Test
include("runtest_taylor.jl")

Test Summary:      | Pass  Total
  pruebas realizadas |    7      7


Base.Test.DefaultTestSet("pruebas realizadas",Any[Test Passed
  Expression: taylor(1,[1]) + taylor(2,[1,2]) == Taylor(1,[2,2])
   Evaluated: Taylor{Int64}(1,[2,2]) == Taylor{Int64}(1,[2,2]),Test Passed
  Expression: taylor(2,[1,1]) + taylor(2,[1,1]) == Taylor(1,[2,2])
   Evaluated: Taylor{Int64}(1,[2,2]) == Taylor{Int64}(1,[2,2]),Test Passed
  Expression: taylor(1,[1]) - taylor(2,[1,2]) == Taylor(1,[0,-2])
   Evaluated: Taylor{Int64}(1,[0,-2]) == Taylor{Int64}(1,[0,-2]),Test Passed
  Expression: taylor(2,[1,1]) - taylor(2,[1,1]) == Taylor(1,[0,0])
   Evaluated: Taylor{Int64}(1,[0,0]) == Taylor{Int64}(1,[0,0]),Test Passed
  Expression: 3 * taylor(3,[10,20,30]) == Taylor(2,[30,60,90])
   Evaluated: Taylor{Int64}(2,[30,60,90]) == Taylor{Int64}(2,[30,60,90]),Test Passed
  Expression: taylor(4,[2,4,6,8]) / 2 == Taylor(3,[1,2,3,4])
   Evaluated: Taylor{Float64}(3,[1.0,2.0,3.0,4.0]) == Taylor{Int64}(3,[1,2,3,4]),Test Passed
  Expression: taylor(2,[1,2]) == Taylor(1,[1,2])
   Evaluated: Taylor

---

## Ejercicio 2

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas necesarias en `runtest_taylor.jl`.